<a href="https://colab.research.google.com/github/shurm/Harry-Potter-Experiments/blob/master/Harry_Potter_Character_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/"
!ls


Stanford NER Tagger. Seems slightly more accurate then nltk pos_tag, but ITS REALLY SLOW.

In [0]:
"""
!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
!unzip stanford-ner-2018-10-16.zip
from nltk.tag.stanford import StanfordNERTagger


st = StanfordNERTagger('/content/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/content/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')
text = None

with open('harry_potter_fan_1.txt', 'r') as file:
    text = file.read()

characters = set({})
for sent in nltk.sent_tokenize(text):
    tokens = nltk.tokenize.word_tokenize(sent)
    tags = st.tag(tokens)
    for tag in tags:
        if tag[1]=='PERSON':
            characters.add(tag)
print("The following characters were found:")
print(characters)

st = StanfordNERTagger('stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')
tokenized_sents = [[word_tokenize(sent) for sent in sent_tokenize(text)] for filename in filelist]
st.tag_sents(tokenized_sents)
print("done")
"""


/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


LookupError: ignored

In [0]:
# Up here, we import additional functionality that we'll need to do this demo.

# The Natural Language Processing Toolkit (NLTK) is a Python library with a lot
# of really powerful tools for textual analysis.
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords 
# collections is a Python library with the super-awesome Counter, which takes a
# list and returns a dictionary that tallies up how many times each value appears.
# For example, ['red', 'red', 'rose'] would become [('red',  2), ('rose': 1)}.
from collections import Counter
# PrettyPrinter will make our final output easier to read in the console.
import pprint

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 
  
story_delim = "chapter 1"
harry_potter_file = open('harry_potter_fan_1.txt', 'r')
#the first line is just "chapter 1"
harry_potter_file.readline() 

#reads the large file one story at a time to prevent memory overflow
def read_story():
    '''
    This function reads the text into python from a text file.
    When you read in your own file, replace 'corpus/hp1.txt' with the path
    to your file.
    '''
    story = []
    while True: 
    
      # Get next line from file 
      line = harry_potter_file.readline() 
      
      # if line is empty 
      # end of file is reached 
      if not line: 
          break

      if line.lower().strip() == story_delim:
          break
      
      story.append(line)
    
    return " ".join(story)




def tagging(tokenize):
    '''
    This function takes the tokenized text created
    by the text_tokenize function and tags each word with a code for the part of speech it represents
    using NLTK's algorithm.  So, it changes the tokenized output:
    ['Harry', 'hung', 'back', 'for', 'a', 'last', 'word',
    'with', 'Ron', 'and', 'Hermione', '.']
    - TO -
    [('Harry', 'NNP'),
    ('hung', 'VBD'), ('back', 'RP'), ('for', 'IN'), ('a', 'DT'), ('last', 'JJ'),
    ('word', 'NN'), ('with', 'IN'), ('Ron', 'NNP'), ('and', 'CC'),
    ('Hermione', 'NNP'), ('.', '.')]
    '''
    tagged_text = pos_tag(tokenize)
    return tagged_text

def find_proper_nouns(tagged_text):
    '''
    This function takes in the tagged text from the tagging function and Returns
    a list of words that were tagged as proper nouns.  It does this by looking
    at the second value in each word/tag pair - e.g. ('Harry', 'NNP') and determining
    if is is equal to 'NNP'.

    There are a lot of characters in these novels who are referred to with two
    proper nouns, like 'Professor Quirell', 'Mrs. Weasley', or 'Uncle Vernon',
    and any character can be called by their full name (e.g. 'Hermione Granger').

    So, if the second value IS equal to 'NNP', we check the second value of the
    next word - if it is also equal to 'NNP', we string the two words together
    and add them to the proper_nouns list.

    If the second value ISN'T equal to 'NNP', we append (add) only the first
    word to the proper_nouns list.

    As we add nouns to the list, we put them all in lower case - otherwise, our
    program won't know that 'HARRY' is the same thing for our purposes as 'Harry'.
    '''
    proper_nouns = []
    i = 0
    while (i+1) < len(tagged_text):
        if tagged_text[i][1] == 'NNP':
            if tagged_text[i+1][1] == 'NNP':
                proper_nouns.append(tagged_text[i][0] +
                                    " " + tagged_text[i+1][0])
                i+=1 # extra increment added to the i counter to skip the next word
            #else:
            #proper_nouns.append(tagged_text[i][0])
        i+=1 # increment the i counter

#    if (i+1)==len(tagged_text):
#        if tagged_text[i][1] == 'NNP':
#            proper_nouns.append(tagged_text[i][0])
    return proper_nouns

def summarize_text(proper_nouns, top_num):
    '''
    This function takes the proper_nouns from the list created by the
    find_proper_nouns function and counts the instances of each.  For this demo,
    we are using the most_common method that comes with the Counter.
    '''
    counts = dict(Counter(proper_nouns).most_common(top_num))
    return counts

# This is where we call all of our functions and pass what they return to the
# next function

num_characters = 100
story_num = 1
most_common_characters = Counter()
while True:
    story = read_story()
    print("story")
    if len(story.strip())==0:
        break
    tokenized_text = word_tokenize(story)
    print("word_tokenize")
    #fliter out
    tokenized_text = [w for w in tokenized_text if w.lower() not in stop_words] 
    print("stop")
    tagged_text = pos_tag(tokenized_text)
    print("pos_tag")
    proper_nouns_in_story = set(find_proper_nouns(tagged_text))
    most_common_characters.update(proper_nouns_in_story)
    print(story_num)
    story_num+=1

#close the file
harry_potter_file.close()

output = most_common_characters.most_common(num_characters)
for o in output:
    print(o)

"""
story_test = "Harry needs to prepare the potions for tomorrow's class."
tokenized_text = word_tokenize(story_test)
tokenized_text = [w for w in tokenized_text if not w in stop_words] 
tagged_text = pos_tag(tokenized_text)
print(tagged_text)
"""

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
story
word_tokenize
stop
pos_tag
1
story
word_tokenize
stop
pos_tag
2
story
word_tokenize
stop
pos_tag
3
story
word_tokenize
stop
pos_tag
4
story
word_tokenize
stop
pos_tag
5
story
word_tokenize
stop
pos_tag
6
story
word_tokenize
stop
pos_tag
7
story
word_tokenize
stop
pos_tag
8
story
word_tokenize
stop
pos_tag
9
story
word_tokenize
stop
pos_tag
10
story
word_tokenize
stop
pos_tag
11
story
word_tokenize
stop
pos_tag
12
story
word_tokenize
stop
pos_tag
13
story
word_tokenize
stop
pos_tag
14
story
('Harry Potter', 13)
('Great Hall', 12)
('Diagon Alley', 11)
('Ron Harry', 11)
('Fred George', 11)
('Ron Hermione', 10)
('Hermione Granger', 9)
('Harry Ginny', 9)
('Mr. Weasley', 9)
('Mrs. Wea

'\nstory_test = "Harry needs to prepare the potions for tomorrow\'s class."\ntokenized_text = word_tokenize(story_test)\ntokenized_text = [w for w in tokenized_text if not w in stop_words] \ntagged_text = pos_tag(tokenized_text)\nprint(tagged_text)\n'

In [0]:
#!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
#!unzip stanford-ner-2018-10-16.zip
from nltk.tag.stanford import StanfordNERTagger


st = StanfordNERTagger('/content/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/content/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

#print(output)
output2 = [w[0] for w in output] 
tags = st.tag(output2)
for tag in tags:
    print(tag)
print(tags)

/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


('Harry', 'LOCATION')
('Potter', 'LOCATION')
('Great', 'LOCATION')
('Hall', 'LOCATION')
('Diagon', 'LOCATION')
('Alley', 'LOCATION')
('Ron', 'PERSON')
('Harry', 'PERSON')
('Fred', 'PERSON')
('George', 'PERSON')
('Ron', 'PERSON')
('Hermione', 'PERSON')
('Harry', 'PERSON')
('Ginny', 'PERSON')
('Mr.', 'O')
('Weasley', 'PERSON')
('Mrs.', 'O')
('Weasley', 'PERSON')
('Harry', 'PERSON')
('Ron', 'PERSON')
('Professor', 'PERSON')
('McGonagall', 'PERSON')
('Hermione', 'PERSON')
('Granger', 'PERSON')
('Draco', 'PERSON')
('Malfoy', 'PERSON')
('Death', 'O')
('Eater', 'O')
('Dark', 'O')
('Lord', 'O')
('Ministry', 'O')
('Magic', 'O')
('Head', 'O')
('Boy', 'O')
('Minister', 'O')
('Magic', 'O')
('Death', 'O')
('Eaters', 'O')
('Hogwarts', 'ORGANIZATION')
('Express', 'ORGANIZATION')
('Pansy', 'ORGANIZATION')
('Parkinson', 'ORGANIZATION')
('James', 'ORGANIZATION')
('Potter', 'ORGANIZATION')
('Hermione', 'ORGANIZATION')
('Ron', 'ORGANIZATION')
('Harry', 'ORGANIZATION')
('Hermione', 'ORGANIZATION')
('Remus'